# Lec 05 基于深度学习的ChatBot

- <a href='#bot'>1. 更聪明的聊天机器人</a>
    - <a href='#bot1'>1.1 生成式模型 VS 检索匹配模型</a>
    - <a href='#bot2'>1.2 Chatterbot的进化：深度学习提高智能度</a>  

- <a href='#model'>2. 模型构建</a>
    - <a href='#model1'>2.1 问题的分析与转化</a>
    - <a href='#model2'>2.2 数据集与样本构造方法</a>
    - <a href='#model3'>2.3 网络结构的构建</a>
    - <a href='#model4'>2.4 模型的评估</a>
    - <a href='#model5'>2.5 代码实现与解析</a>

---

# <a name='bot'>1. 更聪明的聊天机器人</a>


## <a name='bot1'>1.1 生成式模型 VS 检索匹配模型</a>


- 基于检索的chatbot
- 基于生成模型的chatbot

<img src='./images/retrieval_based.png' width='70%'/>

用户输入“query”，根据已有的知识库，在里面检索，匹配到最可能的结果reponse给用户。


问题在于：  
不能生成不存在的内容。  
缺少会话的概念，不支持智能对话。  

<img src='./images/generative_model.png' width='100%'/>

比较复杂，自己生成回复。 

基于序列到序列的模型生成回复，可能会产出“非人类”的回复。

### 聊天机器人的一些思考：
- 基于检索的chatbot
    - 根据input和context，结合知识库的算法得到合适回复
    - 从一个固定的数据集中找到合适的内容作为回复
    - 检索和匹配的方式有很多种
    - 数据和匹配方法对质量有很大影响

- 基于生成模型的chatbot:
    - 典型的是seq2seq的方法
    - 生成的结果需要考虑通畅度和准确度
   

- 以基于检索的chatbot为主（可控度高），生成式模型的chatbot为辅。

- 深度学习发挥什么作用？
    - 需要算法的地方就可以考虑深度学习的优势。

Tricks：
如果有一部分的特征很重要，不希望和一般的特征在深度学习中交叉的学习。

在Tensorflow中有一个wide&deep的模型框架，前面是正常的神经网络模型，在最后一层的时候，可以将最后一层的神经元和这部分不参与神经网络训练的特征向量和这部分神经元的向量做一个拼接，通过逻辑回归等常见的算法最终得到想要的结果。

<img src='./images/trick1.png' width='10%'/>

## <a name='bot2'>1.2 Chatterbot的进化：深度学习提高智能度</a>  

回顾chatterbot：
<img src='./images/chatterbot1.png' width='70%'/>

- 机器人应答逻辑 => Logic Adapters
    - Closest Match Adapter
        - 字符串模糊匹配（编辑距离）
    - Closest Meaning Adapter
        - 借助nltk的WordNet，近义词评估
    - Time Logic Adapter
    - Math Logic Adapter
    - ...

- 应答模式的匹配方式太粗暴
    - 编辑距离无法捕获深层语义信息
    - 核心词+word2vec无法捕获整句话的语义
    
 
- LSTM等RNN模型能捕获序列信息
    - 用深度学习来提高匹配阶段准确率！

# <a name='model'>2. 模型构建</a>

## <a name='model1'>2.1 问题的分析与转化</a>

### 应该怎么做？

“匹配”=>“排序”=>“分类”

- 匹配本身是一个模糊的场景
    - 转成排序问题
   

- 排序问提怎么处理？
    - 转成能输出概率01分类问题
    
分类问题是可以转换成概率向量的，通过转换成分类，就能转换为概率的大小排序了。

- 数据构建？
    - 我们需要正样本（正确的回答）和负样本（不对的回答）

- Loss Function?
    - 这里是01分类问题，回忆一下Logistic Regression，对数损失/二元的交叉熵损失。

在机器学习中，**定义问题**和搭建模型学习同样重要。

### 用深度学习来完成

<img src='./images/retrieval_based_model.png' width='90%'/>

基于检索的机器人，但是里面用到了深度学习。

里面的逻辑是：
上面是Query的文本的序列，下面是Answer的文本序列。  

$c$是捕捉了Query的信息向量，假设为256x1的向量；  
$M$是需要学习的权重系数矩阵，假设为256x256；  
$r$是捕捉了answer的信息向量，假设为256x1的向量。 

$\sigma(c^T M r)$得到的是1x256 x 256x256 x 256x1 ，得到1维的标量，是一个值，通过sigmoid激活函数，将值映射到0-1之间的概率值。

注意：  
上下的两个RNN是相同的，都是从文本中抽取信息，用一套结构就可以了。  
输入的词向量都是词嵌入。  
损失函数就是对数损失（二元的交叉熵）。
$$Loss = -y \ln y' + (1-y)\ln (1-y')$$

## <a name='model2'>2.2 数据集与样本构造方法</a>

### 关于数据

- Ubuntu对话语料库
    - 训练集：
        - Ubuntu对话数据集，来自Ubuntu的IRC网络上的对话日志
        - 训练集1000000条实例，一般是正例(label为1），一半是负例(label为0，负例为随机生成）
        - 样本包括上下文信息（context，即Query）和一段可能的回复内容，即Response; Label为1表示Response和Query的匹配，Label为0则表示不匹配。
        - query的平均长度为86个word，而response的平均长度为17个word

- 训练集：

<img src='./images/ubuntu_corpus.png' width='90%'/>

- 验证/测试集：
    - 每个样本，有一个正例和九个负例数据（也称为干扰数据）。
    - 建模的目标在于给正例的得分尽可能的高，而给负例的得到尽可能的低。（有点类似分类任务）。
    - 语料做过分词、stemmed、lemmatized等文本预处理。
    - 用NER(命名实体识别)将文本中的实体，如：姓名、地点、组织、URL等替换成特殊字符。

 命名实体是一种很特殊的表达， 将其做过处理后，准确率会有一定的上升。

<img src='./images/data.png' width='70%'/>

<a href='./chatbot_dl_retrieval/data_exploration.ipynb'>Ubuntu数据集的基本信息查看代码</a>

## <a name='model3'>2.3 网络结构的构建</a>


## <a name='model4'>2.4 模型的评估</a>


评估准则：
- recall@k
    - 常见的Kaggle比赛评判准则
    - 经模型对候选的response排序后，前k个候选中存在正例数据（正确的那个）的占比
    - k值越大，指标值越高，对模型性能的要求越松。

<img src='./images/recall.png' width='70%'/>

## <a name='model5'>2.5 代码实现与解析</a>